In [1]:
import os
import shutil
import random
import json
import pretty_midi

import numpy as np
import tensorflow as tf
from tensorflow import keras

import pydub
from pydub import AudioSegment

import librosa
import random

import subprocess

import parse

In [2]:
# load data list
data_list = {}
train_list = []
test_list = []

with open(r'data\1. 400-100 dataset\list.json', 'r') as f:
    data_list = json.load(f)

train_list = data_list['train']
test_list = data_list['test']

with open(r'data\0. DEAM\annotations\arousal\arousal.json','r') as load_f:
    arousal_dict = json.load(load_f)
    
with open(r'data\0. DEAM\annotations\valence\valence.json','r') as load_f:
    valence_dict = json.load(load_f)

In [3]:
# load best model
model = keras.models.load_model(r'data\2. Best Model\best_crnn_mid')

In [4]:
# dir
config = r'cat-mel_2bar_small'
train_dir = r'data\1. 400-100 dataset\train'
output_dir = r'data\3-2. Output from MusicVAE'
checkpoint_file = r'data\3-1. MusicVAE checkpoint\train'

In [5]:
# run code in CMD and will print the output of IN TIME ref.: https://lumeng.blog.csdn.net/article/details/104845611
def runCMD(cmd):
    """
    This function will run cmd in jupyter and print the output of cmd on time.
    
    Parameter Description:
    cmd: cmd code.
    """
    screenData = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    while True:
        line = screenData.stdout.readline()
        print(line.decode('gbk').strip("b'"))
        if line == b'' or subprocess.Popen.poll(screenData) == 0:
            screenData.stdout.close()
            break

In [6]:
# split midi in to time fragment
def split_midi_by_time(midi_data,frag_start,frag_finish):
    """
    This function will get the time fragment of a midi data.
    
    Parameter Description:
    midi_data: one midi data of Pretty_MIDI.
    frag_start: the start second of the fragment(include).
    frag_finish: the finish second of the fragment(not include).
    """
    new_midi = midi_data

    for i in range(len(new_midi.instruments)):
        inst = new_midi.instruments[i]
        for n in range(len(inst.notes)-1,-1,-1):
            note = inst.notes[n]
            if ((note.end<frag_start) or (note.start>frag_finish)):
                inst.notes.remove(inst.notes[n])
        for n in range(len(inst.notes)-1,-1,-1):
            note = inst.notes[n]
            note.start = max(note.start - frag_start,0)
            note.end = min(note.end - frag_start, frag_finish - frag_start)

    return new_midi

In [7]:
# interpolate music
def interpolate_music(train_dir,output_dir,music1,music2,frag1,frag2):
    """
    This function will using the 2 4-seconds fragments of 2 music and generate a new music.
    
    Parameter Description:
    train_dir: the folder where the midi file is.
    output_dir: the folder for temp midi file.
    music1: the file name of first music.
    music2: the file name of second music.
    frag1: the start time of the fragment of music1.
    frag2: the start tume of the fragment of music2.
    """
    midi_data = midi_data = pretty_midi.PrettyMIDI(train_dir + '\\' + music1 + '.mp3.mid')
    new_midi = split_midi_by_time(midi_data,(frag1/2+15),(frag1/2+15)+4)
    interMusic_1 = output_dir + '\\'+ music1 + '_trim.mid'
    new_midi.write(interMusic_1)
    
    midi_data = midi_data = pretty_midi.PrettyMIDI(train_dir + '\\' + music2 + '.mp3.mid')
    new_midi = split_midi_by_time(midi_data,(frag2/2+15),(frag2/2+15)+4)
    interMusic_2 = output_dir + '\\'+ music2 + '_trim.mid'
    new_midi.write(interMusic_2)
    
    music_interpolate = r'music_vae_generate \
    --config={config} \
    --checkpoint_file="{checkpoint_file}" \
    --mode=interpolate \
    --num_outputs={num_outputs} \
    --input_midi_1="{interMusic_1}" \
    --input_midi_2="{interMusic_2}" \
    --output_dir="{output_dir}"'

    runCMD(music_interpolate.
              format(config=config,checkpoint_file=checkpoint_file,num_outputs=8,
                    interMusic_1=interMusic_1, interMusic_2=interMusic_2,output_dir=output_dir))

    output_num = 0
    for parent,dirnames, filenames in os.walk(output_dir):
        for filename in filenames:
            if filename.find("cat-mel_2bar_small")!=-1:
                newname = "{music1}-{frag1}th-{frag14}th_{music2}-{frag2}th-{frag24}th_{output_num}-of-2".format(
                    music1=music1,frag1=frag1,frag14=frag1+8,music2=music2,frag2=frag2,frag24=frag2+8,
                        output_num=output_num)
                new_midi_name = os.path.join(parent,newname + ".mid")
                new_wav_name = os.path.join(parent,newname + ".wav")
                
                os.rename(os.path.join(parent,filename),new_midi_name)
                output_num+=1
                
                runCMD(r'"external lib\TiMidity\timidity.exe" "{midifile}" -Ow -o "{wavfile}"'.format(
                    midifile = new_midi_name, wavfile = new_wav_name))
        
        
    os.remove(interMusic_1)
    os.remove(interMusic_2)
    
    return newname[:-7]

In [8]:
# run the model on the output.wav
def evaluate_wav_output(model,output_dir,inter_result):
    """
    This funcion will evaluate the interpolation result by one model.
    
    Parameter Descriptoin:
    model: the folder where the model is.
    output_dir: the folder where the result is.
    inter_result: the name of the interpolation result.
    """
    result = []
    for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
        for filename in filenames:
            if (filename.endswith(".wav") and (filename.find(inter_result)!=-1)):
                index = os.path.splitext(filename)[0]
#                 print(os.path.join(output_dir, filename))
                wavsong = AudioSegment.from_wav(os.path.join(output_dir, filename))

                one_data = []
                test_data = []

                # mel freq
                for i in range(4):
                    outputpath = os.path.join(output_dir,index+'-'+str(i)+".wav")
                    wavsong[(500*i):(500+500*i)].export(outputpath, format="wav")
                    y, sr = librosa.load(outputpath, sr=None)
                    os.remove(outputpath)

                    fragment = np.array(
                                    np.log(
                                        librosa.feature.melspectrogram(
                                            y, sr, n_mels=128,n_fft=int(sr*0.5), hop_length=int(sr*1)
                                        )
                                    ).transpose())
                    fragment = fragment[np.newaxis,:]

                    if one_data==[]:
                        one_data = fragment.copy()                    
                    else:
                        one_data = np.vstack([one_data,fragment.copy()])

                # self repliction
                for i in range(15):
                    if test_data==[]:
                        test_data = one_data.copy()                    
                    else:
                        test_data = np.vstack([one_data,test_data.copy()])                    

                # crnn test
                test_data = test_data.reshape(1,60,128,1)
                predict_va = model.predict(test_data)
                result.append([sum(sum(predict_va[0])/60),sum(sum(predict_va[1])/60)])
    return result

In [9]:
# write result into csv file
def write_result(record_csv,inter_result,eva_result):
    """
    This function will record the result into a csv.
    
    Paramater Description:
    record_csv: the full direction of the record csv.
    inter_result: A string, the name of the interpolation result. To parse this string, we can learn what this result is interpolated from.
    eva_result: A list of pair of real numbers, the evalation result of the interpolation result.
    """
    aver_eva_result = [0,0]
    for i in range(len(eva_result)):
        for j in range(2):
            aver_eva_result[j] += eva_result[i][j]
    aver_eva_result = [i/len(eva_result) for i in aver_eva_result]
    print(aver_eva_result)
    
    inter_result_format = "{music1}-{frag1}th-{frag14}th_{music2}-{frag2}th-{frag24}th"
    parsed = parse.parse(inter_result_format,inter_result)
    
    # music1
    aver_va_music1 = [0,0]
    frag1 = int(parsed['frag1'])
    music1 = int(parsed['music1'])
    for i in range(frag1,frag1+8):
        aver_va_music1[0]+=valence_dict[str(music1)][str(i)]
        aver_va_music1[1]+=arousal_dict[str(music1)][str(i)]
    aver_va_music1[0]/=8
    aver_va_music1[1]/=8
    
    aver_va_music2 = [0,0]
    frag2 = int(parsed['frag2'])
    music2 = int(parsed['music2'])
    for i in range(frag2,frag2+8):
        aver_va_music2[0]+=valence_dict[str(music2)][str(i)]
        aver_va_music2[1]+=arousal_dict[str(music2)][str(i)]
    aver_va_music2[0]/=8
    aver_va_music2[1]/=8
    
    print(aver_va_music1,aver_va_music2)
    
    with open(record_csv,"a+") as f:
        f.write("{0},{1},{2},{3},{4},{5},{6}\n".format(
                inter_result,
                aver_va_music1[0],aver_va_music1[1],aver_va_music2[0],aver_va_music2[1],
                aver_eva_result[1],aver_eva_result[0]))
        # NOTE: The first is VOLENCE and the second is Arousal in the output of the model.

In [10]:
with open("external lib\TiMidity\TIMIDITY.cfg","w+") as f:
    f.write("soundfont \"{}\\external lib\\TiMidity\\GeneralUser.SF2\"".format(os.getcwd()))

In [15]:
# test several times to find how labels changing
testtimes = 10000
for i in range(testtimes):
    [music_1,music_2] = random.sample(train_list,2)
    frag_1 = random.randint(0,52)
    frag_2 = random.randint(0,52)
    inter_result = interpolate_music(train_dir,output_dir,music_1,music_2,frag_1,frag_2)

    eva_result = evaluate_wav_output(model,output_dir,inter_result)

    print(inter_result,eva_result)
    write_result(r"data\3-3. Emotion Changing result\label_changing.csv",inter_result,eva_result)


Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1105 22:55:14.837425 10200 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

2 valid inputs extracted from `data\3-2. Output from MusicVAE\1088_trim.mid`. Outputting these potential inputs as `data\3-2. Output from MusicVAE\cat-mel_2bar_small_input1-extractions_2020-11-05_225514-*-of-002.mid`. Call script again with one of these instead.


Playing data\3-2. Output from MusicVAE\1088-32th-40th_461-19th-27th_0-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1088-32th-40th_461-19th-27th_0-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~8 seconds

Notes cut: 0

Notes lost totally: 0


Playing data\3-2. Output from MusicVAE\1088-32th-40th_461-19th-27th_1-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1088-32th-40th_46

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


1088-32th-40th_461-19th-27th [[0.033670645745587535, -0.31702945567667484], [0.06686757401621435, -0.2467926059034653]]
[0.05026910988090094, -0.28191103079007007]
[0.5663750000000001, 0.46962499999999996] [-0.06842697500000001, 0.06537743875]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1105 22:55:24.791019  6100 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

2 valid inputs extracted from `data\3-2. Output from MusicVAE\1476_trim.mid`. Outputting these potential inputs as `data\3-2. Output from MusicVAE\cat-mel_2bar_small_input1-extractions_2020-11-05_225524-*-of-002.mid`. Call script again with one of these instead.


Playing data\3-2. Output from MusicVAE\1476-6th-14th_800-18th-26th_0-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1476-6th-14th_800-18th-26th_0-of-2.mid



Playing data\3-2. Output from MusicVAE\585-1th-9th_657-47th-55th_0-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\585-1th-9th_657-47th-55th_0-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~10 seconds

Notes cut: 0

Notes lost totally: 0


Playing data\3-2. Output from MusicVAE\585-1th-9th_657-47th-55th_1-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\585-1th-9th_657-47th-55th_1-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~8 seconds

Notes cut: 0

Notes lost totally: 0


585-1th-9th_657-47th-55th [[0.21371741349139484, -0.11623414677234223], [0.1522332185995765, -0.12228096089165774]]
[0.18297531604548567, -0.11925755383199999]
[-0.16058687375000003, -0.16175871] [0.039958742, 0.17403015000000002]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1105 22:56:27.404816  4664 music_vae_generate.py:145] Atte


Instructions for updating:

Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API

W1105 22:56:49.519562 13528 deprecation.py:323] From c:\users\arnold\desktop\4560\musicvae\magenta\magenta\contrib\rnn.py:474: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.

Instructions for updating:

Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


Instructions for updating:

Please use `keras.layers.RNN(cell)`, which is equivalent to this API

W1105 22:56:49.519562 13528 deprecation.py:323] From E:\anaconda3\lib\site-packages\tensorflow_core\python\ops\rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.

Instructions for updating:

Please use `keras.layers.RNN(cell)`, which is equivalent to this API

2020-11-05 22:56:49.962270: I tensorflow/core/platform/cpu_feature_guard.cc:142] You

...


Playing data\3-2. Output from MusicVAE\1110-49th-57th_1798-52th-60th_0-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1110-49th-57th_1798-52th-60th_0-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~10 seconds

Notes cut: 0

Notes lost totally: 0


Playing data\3-2. Output from MusicVAE\1110-49th-57th_1798-52th-60th_1-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1110-49th-57th_1798-52th-60th_1-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~8 seconds

Notes cut: 0

Notes lost totally: 0


1110-49th-57th_1798-52th-60th [[0.022834231366687163, -0.17612647684291005], [0.23973743220994947, -0.16246273089200258]]
[0.13128583178831832, -0.16929460386745632]
[0.05525000000000001, 0.47500000000000003] [0.235, 0.005875]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1106 00:47:19.074094 13896 music_vae_generate.

Playing time: ~10 seconds

Notes cut: 0

Notes lost totally: 0


1090-6th-14th_1057-19th-27th [[0.1177393150574062, -0.19403704878641292], [0.01687360994401388, -0.2505651486571878]]
[0.06730646250071004, -0.22230109872180037]
[0.34199999999999997, 0.52775] [-0.012875000000000001, 0.6996249999999999]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1106 00:48:11.346966  9908 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

2 valid inputs extracted from `data\3-2. Output from MusicVAE\500_trim.mid`. Outputting these potential inputs as `data\3-2. Output from MusicVAE\cat-mel_2bar_small_input1-extractions_2020-11-06_004811-*-of-002.mid`. Call script again with one of these instead.


Playing data\3-2. Output from MusicVAE\500-22th-30th_1246-47th-55th_0-of-2.mid

MIDI file: data\3-2. Ou


Playing data\3-2. Output from MusicVAE\1153-11th-19th_1310-32th-40th_0-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1153-11th-19th_1310-32th-40th_0-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~10 seconds

Notes cut: 0

Notes lost totally: 0


Playing data\3-2. Output from MusicVAE\1153-11th-19th_1310-32th-40th_1-of-2.mid

MIDI file: data\3-2. Output from MusicVAE\1153-11th-19th_1310-32th-40th_1-of-2.mid

Format: 1  Tracks: 2  Divisions: 220

Playing time: ~8 seconds

Notes cut: 0

Notes lost totally: 0


1153-11th-19th_1310-32th-40th [[0.18903437681456126, -0.27593475161120296], [0.2347916069556959, -0.18954606272745878]]
[0.21191299188512858, -0.23274040716933087]
[-0.22325000000000003, -0.099875] [0.080625, 0.121125]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1106 00:49:14.201028  3756 music_vae_generate.py:145] 

Playing time: ~10 seconds

Notes cut: 0

Notes lost totally: 0


1932-17th-25th_195-12th-20th [[0.0031152301371548674, -0.2284457548521459], [0.0438733728697116, -0.1833324676263146]]
[0.023494301503433235, -0.20588911123923026]
[-0.12474999999999999, 0.54775] [-0.35633719999999997, -0.245157]

Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1106 00:50:08.353240 13816 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

2 valid inputs extracted from `data\3-2. Output from MusicVAE\1211_trim.mid`. Outputting these potential inputs as `data\3-2. Output from MusicVAE\cat-mel_2bar_small_input1-extractions_2020-11-06_005008-*-of-002.mid`. Call script again with one of these instead.


Playing data\3-2. Output from MusicVAE\1211-19th-27th_67-14th-22th_0-of-2.mid

MIDI file: data\3-2. Output fr

UnboundLocalError: local variable 'newname' referenced before assignment